### What is Reactive programming?

Reactive programming is programming with asynchronous data streams. A stream is a sequence of ongoing events ordered in time.

Here is the reactive terminology in simple terms:

- __Observable__ - An _array_ that is built over time.
- __Subscription__ - A _for loop_ that happens over time.
- __Emitting__ - When an Observable spits out a value because something subscribed to it.
- __Operator__ - Functions that create a new observable from existing observables.

### Excellent Recources

- [RxMarbles](https://rxmarbles.com) - A Visual Guide to Rx
- [RxJS](https://rxjs.dev) - RxJS (reactive extensions library for JavaScript) documentation
- [RxJS Quick Start](https://angularfirebase.com/lessons/rxjs-quickstart-with-20-examples/) - Quick start for RxJS (this python notebook is an adaptaion of this article)

### Create Observables - Define the stream

There are many ways to create __Observables__. Remember, a stream can be anything. The most fundamental method is to use _creation operators_.

In [2]:
using Rocket

array_source = from(1:100)

subscription = subscribe!(array_source |> sum(), lambda(
    on_next = (d) -> println(d)
))

unsubscribe!(subscription)

5050


In [3]:
never_source  = never(Int)
error_source  = faulted(Int, "Some error")
file_source   = file("data")
single_source = of(0)
timer_source  = timer(100, 100);

There are two main types of the Observable.

- __Cold Observables__ start emitting or creating values only when the subscription starts, like a typical YouTube video. Each subscriber will see the same sequence (or pattern) of events from start to finish.
- __Hot Observables__ are always being updated with new values, like a live stream on YouTube. When you subscribe you start with the most recent value and only see future changes.

This is somewhat confusing topic that should be really simple. Check out Ben Lesh’s post on [Hot vs. Cold Observables on Medium](https://medium.com/@benlesh/hot-vs-cold-observables-f8094ed53339).

__Rocket.jl__ also provides a general _creation operator_ which allows you to emit values to the subscriber by calling `next!()` in the function. 

The following is an Observable that emits the values 1, 2, 3 immediately (synchronously) when subscribed to, and the value 4 after one second has passed since the subscribe call, then completes:

In [4]:
my_custom_source = make(Int) do actor
    next!(actor, 0)
    next!(actor, 1)
    next!(actor, 2)
    setTimeout(1000) do
        next!(actor, 3)
        complete!(actor)
    end
end;

### Map - Control the Emmited Values

The single most powerful Rocket.jl operator is `map`. It allows you to modify the emitted values based on any logic you desire. It’s the chef’s knife of Rocket.jl operators - you will use it often. Here we have an observable of numbers, let’s map them to their logarithm.

In [5]:
numbers = from([ 10, 100, 1000 ])

subscribe!(numbers |> map(Float64, log10), lambda(
    on_next     = (d) -> println(d),
    on_error    = (e) -> println(e),
    on_complete = ()  -> println("Completed")
));

1.0
2.0
3.0
Completed


### Filter - Use only the Good Stuff

Filter will only emit values that meet a certain condition. Suppose we have an Observable of letters where we are only interested in the uppercase ones. 

In [7]:
letters = from("HhEeLlLlOo, WwOoRrLlDd!")

subscribe!(letters |> filter(isuppercase), lambda(
    on_next = (d) -> print(d)
));

HELLOWORLD

### Tap - Side-effects 

The `tap` operator allows you to run side-effect code while the Observable emits, without producing side effects on the emitted values themself. This is handy for debugging or for any situation where you want to run code outside of the Observable scope.

In [8]:
subscribe!(numbers |> 
    tap((d) -> println("Original: $d")) |> map(Float64, log10) |> 
    tap((d) -> println("Log10: $d")), lambda(on_complete = () -> println("Completed")));

Original: 10
Log10: 1.0
Original: 100
Log10: 2.0
Original: 1000
Log10: 3.0
Completed


### First, Last - When the order counts

There are times when you might only care about the first or last element of an Observable.

In [9]:
names = from([ "Albert", "Wouter", "Ivan", "Dmitry" ])

subscribe!(names |> last(), lambda(
    on_next = (d) -> println(d))
);

Dmitry


In [10]:
names = from([ "Albert", "Wouter", "Ivan", "Dmitry" ])

subscribe!(names |> first(), lambda(
    on_next = (d) -> println(d))
);

Albert


### Scan - Keep a Running Total

Scan is the reactive counterpart for the `reduce` function used with normal arrays. It keeps track of the accumulated total of emitted values. In other words, it allows you to combine emitted values from an observable. Here is a handwritten version of the `enumerate` operator.

In [11]:
obs = from("Hello World!")

subscribe!( obs |> scan(Tuple{Char, Int}, (curr_char, acc_tuple) -> (curr_char, acc_tuple[2] + 1), (' ', 0)), lambda(
    on_next = (d) -> println(d)
));

('H', 1)
('e', 2)
('l', 3)
('l', 4)
('o', 5)
(' ', 6)
('W', 7)
('o', 8)
('r', 9)
('l', 10)
('d', 11)
('!', 12)


### Subject - An Observable that talks to multiple subscribers

A `Subject` is an Observable with the ability to call `next!()` on itself to emit new values - in other words, it is an event emitter.

In [12]:
subject = Subject(Int);

In [13]:
next!(subject, 1);

In [14]:
subscription1 = subscribe!(subject, (d) -> println("[1] ", d));

In [15]:
next!(subject, 2);

[1] 2


In [16]:
subscription2 = subscribe!(subject, (d) -> println("[2] ", d));

In [17]:
next!(subject, 3);

[1] 3
[2] 3


In [18]:
unsubscribe!(subscription1);

In [19]:
next!(subject, 4);

[2] 4


In [20]:
unsubscribe!(subscription2);

In [21]:
next!(subject, 5);

### Actor - A Subscriber which knows what to do

An actor is the primitive unit of computation. It is the entity at the end of the stream that receives events and processes them in a meaningful manner. There are a series of predefined Actors with a certain logic. It is also possible (and this is the preffered way) to define them from scratch. See the documentation for the details.

In [22]:
common_actor = lambda(
    on_next     = (d) -> println(d),
    on_error    = (e) -> println(e),
    on_complete = ()  -> println("Completed")
)

logger_actor   = logger()
nothing_actor  = void();
function_actor = (d) -> println("Data: ", d);

In [23]:
subscribe!(from(1:5), common_actor);

1
2
3
4
5
Completed


In [24]:
subscribe!(from(1:5), logger_actor);

[LogActor] Data: 1
[LogActor] Data: 2
[LogActor] Data: 3
[LogActor] Data: 4
[LogActor] Data: 5
[LogActor] Completed


In [25]:
subscribe!(from(1:5), nothing_actor);

In [26]:
subscribe!(from(1:5), function_actor);

Data: 1
Data: 2
Data: 3
Data: 4
Data: 5


Rocket.jl has a big collection of operators and variations of Observables, but this examples cover a major chunk of the core concepts you need to understand reactive way of programming.